# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [3]:
import pandas as pd
import requests

# 1) Carregar a base e verificar duplicados:

# Carregando a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contando número de registros
print(f"Número total de registros: {sinasc.shape[0]}")

# Removendo duplicados e contando número de registros únicos
sinasc_unique = sinasc.drop_duplicates()
print(f"Número de registros únicos: {sinasc_unique.shape[0]}")

Número total de registros: 27028
Número de registros únicos: 27028


In [4]:
# 2) Contar valores missing por variável:

# Contar valores missing por coluna
missing_values = sinasc.isnull().sum()
print("Valores missing por variável:\n", missing_values)

Valores missing por variável:
 ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [5]:
# 3) Selecionar colunas de interesse e contar valores missing novamente:

# Selecionar colunas de interesse
cols_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[cols_interesse]

# Contar valores missing nas colunas de interesse
missing_values_selecionado = sinasc_selecionado.isnull().sum()
print("Valores missing nas colunas selecionadas:\n", missing_values_selecionado)


Valores missing nas colunas selecionadas:
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [6]:
# 4) Remover registros com APGAR5 não preenchido:

# Remover registros com APGAR5 não preenchido
sinasc_selecionado = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contar o número de linhas após remoção e valores missing
print(f"Número de linhas após remoção de APGAR5 não preenchido: {sinasc_selecionado.shape[0]}")
missing_values_pos_remocao = sinasc_selecionado.isnull().sum()
print("Valores missing após remoção de APGAR5:\n", missing_values_pos_remocao)


Número de linhas após remoção de APGAR5 não preenchido: 26925
Valores missing após remoção de APGAR5:
 LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [7]:
# 5) Tratar os valores 9 como missing para ESTCIVMAE e CONSULTAS:

# Substituir o valor 9 por NaN para ESTCIVMAE e CONSULTAS
sinasc_selecionado['ESTCIVMAE'] = sinasc_selecionado['ESTCIVMAE'].replace(9, pd.NA)
sinasc_selecionado['CONSULTAS'] = sinasc_selecionado['CONSULTAS'].replace(9, pd.NA)



In [8]:
# 6) Substituir valores missing de QTDFILVIVO por zero:

# Substituir valores missing de QTDFILVIVO por zero
sinasc_selecionado['QTDFILVIVO'].fillna(0, inplace=True)



In [9]:
# 7) Decidir sobre outros valores missing e justificativas: Você pode decidir entre substituir os valores missing com o valor mais comum ou mantê-los como NaN. Por exemplo:

# Exemplo: Substituir missing de GESTACAO com a moda (valor mais comum)
gestacao_moda = sinasc_selecionado['GESTACAO'].mode()[0]
sinasc_selecionado['GESTACAO'].fillna(gestacao_moda, inplace=True)


In [10]:
# 8) Criar categorização da variável APGAR5:

# Criar uma categorização para APGAR5
def categorizar_apgar5(apgar5):
    if apgar5 >= 8:
        return 'normal'
    elif apgar5 >= 6:
        return 'asfixia leve'
    elif apgar5 >= 4:
        return 'asfixia moderada'
    else:
        return 'asfixia severa'

sinasc_selecionado['APGAR5_CATEGORIA'] = sinasc_selecionado['APGAR5'].apply(categorizar_apgar5)

# Calcular frequências dessa categorização
freq_apgar5 = sinasc_selecionado['APGAR5_CATEGORIA'].value_counts()
print("Frequência da categorização de APGAR5:\n", freq_apgar5)


Frequência da categorização de APGAR5:
 APGAR5_CATEGORIA
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [11]:
# 9) Renomear variáveis para snake case:

# Renomear as colunas para snake case
sinasc_selecionado.columns = sinasc_selecionado.columns.str.lower().str.replace(' ', '_')



## Conclusão sobre a Análise de Nascidos Vivos em Rondônia (RO) - 2019

Com base nos resultados obtidos, algumas conclusões importantes podem ser tiradas sobre a situação dos nascimentos em Rondônia, levando em consideração a taxa de asfixia neonatal e possíveis fatores de risco:

### 1. Taxa de Asfixia Neonatal

A categorização da variável `APGAR5`, que reflete o estado de saúde dos recém-nascidos 5 minutos após o nascimento, revelou as seguintes frequências:

- **Normal (APGAR 8-10)**: 26.463 casos (98,28%)
- **Asfixia Leve (APGAR 6-7)**: 320 casos (1,19%)
- **Asfixia Moderada (APGAR 4-5)**: 68 casos (0,25%)
- **Asfixia Severa (APGAR 0-3)**: 74 casos (0,27%)

Embora a maioria dos bebês tenha nascido em condições normais, uma pequena porcentagem sofreu asfixia leve a severa. **A maior taxa de mortes de bebês está provavelmente associada aos casos de asfixia severa (0,27%)**, uma vez que esta categoria está fortemente relacionada ao risco de mortalidade neonatal.

### 2. Fatores Possíveis Associados à Asfixia

Alguns fatores que, com base na literatura médica, podem estar relacionados à asfixia neonatal incluem:

- **Gestação Precoce ou Tardia**: Valores missing na variável `GESTACAO` foram 1.216 casos. Prematuridade (gestação abaixo de 37 semanas) é um fator de risco conhecido para complicações como asfixia.
  
- **Número de Consultas Pré-Natal**: A ausência de consultas pré-natais adequadas é outro fator. O pré-natal insuficiente pode levar a partos com complicações. Mães com menos de 4 consultas podem ter maior risco de complicações.

- **Gravidez Múltipla**: A variável `GRAVIDEZ` apresentou 76 valores missing. Gravidezes múltiplas (gêmeos, trigêmeos) têm mais complicações, incluindo maior risco de asfixia.

- **Idade Materna e Condições Socioeconômicas**: Variáveis como `IDADEMAE`, `ESTCIVMAE` (estado civil) e `ESCMAE` (escolaridade) podem influenciar nos resultados. Mães muito jovens ou mais velhas e de baixa escolaridade têm maior chance de complicações no parto.

- **Tipo de Parto**: O tipo de parto (vaginal ou cesáreo) pode afetar os resultados neonatais. Cesáreas de emergência, por exemplo, estão frequentemente associadas a complicações como asfixia.

### 3. Conclusão Geral

Apesar da taxa de asfixia severa ser pequena (0,27%), ela representa um risco considerável para a mortalidade neonatal. Complicações como prematuridade, pré-natal insuficiente, partos múltiplos e condições socioeconômicas desfavoráveis são fatores a serem considerados para mitigar os riscos.

**Recomendações**: Investigar mais profundamente esses fatores e garantir acesso adequado a cuidados de saúde durante a gravidez pode reduzir as taxas de mortalidade e complicações no nascimento.
